In [7]:
import salabim as sim
import pandas as pd
from copy import deepcopy

# --- Main simulation setup ---
env = sim.Environment(trace=False)
sim.yieldless(False)


False

In [9]:
# run_mastercode02.py
import salabim as sim
import pandas as pd
from copy import deepcopy
import traceback
import time

# UPDATED: All imports now refer to mastercode02
from mastercode02_config_and_rates import RATES as BASE_RATES, SIMULATION_DURATION
from mastercode02_agents import (Person, MarriageManager, WorldManager, EconomicManager,
                                 PublicTransitManager, TrafficManager, EducationManager,
                                 GovernmentManager, CommuteManager, ScenarioManager,
                                 HouseholdFinanceManager, ImmigrationManager)
from mastercode02_setup import Initializer
from mastercode02_logging import log_yearly_data, write_and_close_csv_logs
import mastercode02_globals as g

def get_trigger_events():
    """Prompts the user to define the active trigger events for the simulation."""
    events = []
    print("--- Configure Trigger Events ---")

    def get_input(prompt, type_func=int):
        while True:
            try:
                return type_func(input(prompt))
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    if input("Activate Political Stability scenario? (yes/no): ").lower() == 'yes':
        print("\nConfiguring Political Stability...")
        start = get_input("  Enter Start Year: ")
        end = get_input("  Enter End Year: ")
        level = get_input("  Enter Level (1-5): ")
        events.append({
            'type': 'political_stability', 'enabled': True,
            'start_year': start, 'end_year': end, 'level': level
        })

    if input("\nActivate Panic (Disaster/Pandemic) scenario? (yes/no): ").lower() == 'yes':
        print("\nConfiguring Panic Scenario...")
        start = get_input("  Enter Start Year: ")
        end = get_input("  Enter End Year: ")
        level = get_input("  Enter Level (1-5): ")
        events.append({
            'type': 'panic', 'enabled': True,
            'start_year': start, 'end_year': end, 'level': level
        })

    if input("\nActivate Public Health scenario? (yes/no): ").lower() == 'yes':
        print("\nConfiguring Public Health Scenario...")
        start = get_input("  Enter Start Year: ")
        end = get_input("  Enter End Year: ")
        level = get_input("  Enter Level (-5 for crisis to 5 for boom): ")
        events.append({
            'type': 'public_health', 'enabled': True,
            'start_year': start, 'end_year': end, 'level': level
        })
    
    print("\n--- Trigger Event configuration complete. ---")
    return events

class YearlyReporter(sim.Component):
    def setup(self, file_path):
        self.file_path = file_path
        self.last_year_pop = 0

    def process(self):
        yield self.hold(0)
        self.last_year_pop = len(g.POPULATION)
        print("YearlyReporter: Logging initial state at Year 0...")
        log_yearly_data(0, self.env, 0.0)
        while True:
            yield self.hold(1)
            year = int(self.env.now())
            current_pop = len(g.POPULATION)
            growth_rate = (current_pop - self.last_year_pop) / self.last_year_pop if self.last_year_pop > 0 else 0
            self.last_year_pop = current_pop
            print(f"YearlyReporter: Logging end-of-year data for Year {year}...")
            log_yearly_data(year, self.env, growth_rate)

def run_simulation_scenario():
    trigger_events = get_trigger_events()
    
    event_names = "_".join([evt['type'][:4] + str(evt['level']) for evt in trigger_events if evt['enabled']])
    if not event_names: event_names = "baseline"
    base_filename = f'mastercode02_output_{event_names}_{pd.Timestamp.now().strftime("%Y%m%d_%H%M")}'

    env = sim.Environment(time_unit='years', random_seed=123)
    try:
        g.RATES = deepcopy(BASE_RATES)
        env.RATES = g.RATES

        # STAGE 1
        Initializer(env=env)
        ScenarioManager(env=env, events=trigger_events)
        EconomicManager(env=env)
        WorldManager(env=env)

        print("\nInitializer & Managers: Building world and setting initial rates...")
        env.run(till=0)

        # STAGE 2
        PublicTransitManager(env=env)
        MarriageManager(env, g.HOUSEHOLDS, g.POPULATION, g.MARRIAGES)
        TrafficManager(env=env)
        CommuteManager(env=env)
        GovernmentManager(env=env)
        HouseholdFinanceManager(env=env)
        EducationManager(env=env)
        ImmigrationManager(env=env)
        YearlyReporter(env=env, file_path=base_filename)

        # STAGE 3
        print(f"\nStarting {SIMULATION_DURATION}-year simulation with events: {event_names}")
        start_time = time.time()
        env.run(till=SIMULATION_DURATION)
        end_time = time.time()
        print(f"\nSimulation finished in {end_time - start_time:.2f} seconds. Preparing final log files.")

    finally:
        write_and_close_csv_logs(base_filename, env)
        try:
            summary_df = pd.read_csv(f"{base_filename}_annual_summary.csv")
            print("\n--- Final Year Summary (from generated CSV) ---")
            print(summary_df.iloc[-1].to_string())
            print("---------------------------------------------")
        except (FileNotFoundError, pd.errors.EmptyDataError):
            print("\nCould not read summary CSV to print final results (file might be empty or not found).")
        except Exception as e:
            print(f"\nAn error occurred while printing final summary: {e}")
            traceback.print_exc()

if __name__ == '__main__':
    run_simulation_scenario()

--- Configure Trigger Events ---


Activate Political Stability scenario? (yes/no):  yes



Configuring Political Stability...


  Enter Start Year:  8
  Enter End Year:  11
  Enter Level (1-5):  4

Activate Panic (Disaster/Pandemic) scenario? (yes/no):  yes



Configuring Panic Scenario...


  Enter Start Year:  20
  Enter End Year:  22
  Enter Level (1-5):  3

Activate Public Health scenario? (yes/no):  no



--- Trigger Event configuration complete. ---

Initializer & Managers: Building world and setting initial rates...

--- RUNNING MASTERCODE02 INITIALIZER ---

--- Starting Programmatic Population Generation ---
SUCCESS: Generated exactly 73440 temporary agent data objects.
Initializer: Pre-assigning initial employers and income...
Initializer: Initial state setup is complete.

SUCCESS: Initialized and activated 73440 people in 31162 households.

Starting 50-year simulation with events: poli4_pani3
YearlyReporter: Logging initial state at Year 0...
ImmigrationManager: Adding 294 new agents this year.
YearlyReporter: Logging end-of-year data for Year 1...

--- Writing final logs to CSV files with base name: mastercode02_output_poli4_pani3_20251008_1020 ---
...All CSV log files saved successfully. ✅

--- Final Year Summary (from generated CSV) ---
Year                               1.00
Population                     75337.00
Households                     34811.00
Population Growth rate 

KeyboardInterrupt: 

# GRAPHS AND DATA TABLES

In [6]:
import pandas as pd

In [10]:
Pop_data=pd.read_csv("mastercode02_output_poli3_20251007_2026_population_datasheet.csv")
Pop_data

,year,person_id,age,sex,new household_id,household_type,Household Id Before marriage,marital_status,Spouses former Household Id,education_level,...,Emplolyment Rank,employer,Layoff/ dismissed / quit before?,Has a car?,Car ID,Use the Bus?,Accident Involvement,Most used Purpose of comutation,Annual Income,Gov_support_cum per person
0,50,73441,6,Male,48219,Nonfamily household,NaN,Never married,NaN,Elementary (K-5),...,NaN,NaN,False,False,NaN,False,False,Healthcare,0.0,0.0
1,50,73442,52,Male,83308,Married-couple family household,51785.0,Married,33674.0,too_young,...,NaN,NaN,False,True,CAR_70675,False,False,Work,0.0,21000.0
2,50,73443,8,Male,37800,Married-couple family household,NaN,Never married,NaN,Elementary (K-5),...,NaN,NaN,False,False,NaN,False,False,Leisure,0.0,0.0
3,50,73444,9,Male,52521,Nonfamily household,NaN,Never married,NaN,Middle (6-8),...,NaN,NaN,False,False,NaN,False,False,Leisure,0.0,0.0
4,50,73445,52,Male,83286,Married-couple family household,43346.0,Married,38256.0,too_young,...,NaN,NaN,False,False,NaN,False,False,Shopping,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156853,50,248516,31,Male,99308,Nonfamily household,NaN,Never married,NaN,college_completed,...,NaN,NaN,False,False,NaN,False,False,Other,0.0,0.0
156854,50,248517,27,Female,99309,Nonfamily household,NaN,Never married,NaN,college_completed,...,NaN,NaN,False,False,NaN,False,False,Other,0.0,0.0
156855,50,248518,23,Male,99310,Nonfamily household,NaN,Never married,NaN,college_completed,...,NaN,NaN,False,False,NaN,False,False,Other,0.0,0.0
156856,50,248519,23,Male,99311,Nonfamily household,NaN,Never married,NaN,high_school_completed,...,NaN,NaN,False,False,NaN,False,False,Other,0.0,0.0


In [12]:
HH_data=pd.read_csv("mastercode02_output_poli3_20251007_2026_household_datasheet.csv")
HH_data

,household_id,household_type,household_members,births_in_hh,deaths_in_hh,marriages_in_hh,students_in_hh,employed_in_hh,Have a car?,How many cars in household,use bus?,Household Annual Income,Required Cost_Pre tax,Net_income_minus_cost,Savings Balance,Loan Balance,Loan Repaid_cum,Gov_support_cum,Taxes_cum
0,31163,Married-couple family household,1,0,0,0,1,1,True,1,False,60013.38,37532.78,22480.59,54340.34,833.42,85783.99,0.0,1047736.71
1,31164,Married-couple family household,1,0,0,0,0,0,False,0,True,0.00,64404.00,-64404.00,0.00,2891718.20,0.00,0.0,107748.00
2,31167,Married-couple family household,1,0,0,0,0,0,True,2,False,48461.53,37532.78,10928.74,18529.98,815007.30,3577.56,0.0,912808.97
3,31168,Married-couple family household,2,0,0,0,0,0,False,0,False,0.00,79672.57,-79672.57,0.00,3867280.80,0.00,0.0,0.00
4,31169,Married-couple family household,1,0,0,0,1,1,True,1,False,110949.52,37532.78,73416.74,1637062.94,0.00,0.00,0.0,1930910.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46856,99308,Nonfamily household,1,0,0,0,0,0,False,0,False,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
46857,99309,Nonfamily household,1,0,0,0,0,0,False,0,False,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
46858,99310,Nonfamily household,1,0,0,0,0,0,False,0,False,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
46859,99311,Nonfamily household,1,0,0,0,0,0,False,0,False,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00


In [14]:
ANNUAL_data=pd.read_csv("mastercode02_output_poli3_20251007_2026_annual_summary.csv")
ANNUAL_data

,Year,Population,Households,Population Growth rate,No of births,Birth rate,No. of Deaths,Death rate,Employment Rate,Avg Annual Income,Avg Annual Req Cost,Avg Savings Balance,Avg Loan Balance,Total Number of Cars,Number of Accidents on road,Cost Inflation rate
0,0,72641,31162,0.00,0,0.00,799,0.01,0.70,56774.06,0.00,0.00,0.00,0,0,0.04
1,1,75323,34797,0.04,3221,0.04,833,0.01,0.66,59821.81,52168.24,7917.89,10058.34,0,0,-0.01
2,2,79592,37596,0.06,4751,0.06,798,0.01,0.73,62277.18,53476.50,15940.28,18448.37,11325,67,0.02
3,3,84782,39374,0.07,5615,0.07,694,0.01,0.72,64268.90,55445.65,22630.65,26418.60,19838,111,0.04
4,4,89941,40783,0.06,5630,0.07,691,0.01,0.72,66165.24,54319.17,31754.87,33881.32,26241,136,0.00
5,5,95242,42007,0.06,5712,0.06,694,0.01,0.72,67776.00,57420.74,40447.04,40982.33,30894,145,0.04
6,6,99739,43090,0.05,4988,0.05,731,0.01,0.71,68828.98,55560.12,48806.19,47908.23,34128,184,-0.00
7,7,103561,44060,0.04,4125,0.04,612,0.01,0.71,70211.34,58729.75,53867.04,54732.00,36493,173,0.04
8,8,107107,44764,0.03,3986,0.04,697,0.01,0.71,72020.27,59566.25,62044.98,61332.75,38332,189,0.05
9,9,111083,45348,0.04,4500,0.04,788,0.01,0.70,73931.89,59543.54,70154.84,67859.19,39702,593,0.03


In [24]:
Pop_data.info()

HH_data.info()

ANNUAL_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156858 entries, 0 to 156857
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   year                              156858 non-null  int64  
 1   person_id                         156858 non-null  int64  
 2   age                               156858 non-null  int64  
 3   sex                               156858 non-null  object 
 4   new household_id                  156858 non-null  int64  
 5   household_type                    156858 non-null  object 
 6   Household Id Before marriage      36638 non-null   float64
 7   marital_status                    156858 non-null  object 
 8   Spouses former Household Id       36638 non-null   float64
 9   education_level                   156858 non-null  object 
 10  employment_status                 156858 non-null  object 
 11  Emplolyment Rank                  56417 non-null   o

In [26]:
Marriage_data=pd.read_csv("mastercode02_output_poli3_20251007_2026_summary_marriages.csv")
Marriage_data.info()

Rate_data=pd.read_csv("mastercode02_output_poli3_20251007_2026_summary_rates.csv")
Rate_data.info()

Score_data=pd.read_csv("mastercode02_output_poli3_20251007_2026_scores_summary.csv")
Score_data.info()

Resource_data=pd.read_csv("mastercode02_output_poli3_20251007_2026_summary_resources.csv")
Resource_data.info()

Trip_data=pd.read_csv("mastercode02_output_poli3_20251007_2026_summary_trips.csv")
Trip_data.info()

Vehicle_data=pd.read_csv("mastercode02_output_poli3_20251007_2026_summary_vehicle_events.csv")
Vehicle_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19667 entries, 0 to 19666
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   marriage_id       19667 non-null  object
 1   man_id            19667 non-null  int64 
 2   man_former_hh     19667 non-null  int64 
 3   man_age           19667 non-null  int64 
 4   woman_id          19667 non-null  int64 
 5   woman_former_hh   19667 non-null  int64 
 6   woman_age         19667 non-null  int64 
 7   new_hh_id         19667 non-null  int64 
 8   year_of_marriage  19667 non-null  int64 
dtypes: int64(8), object(1)
memory usage: 1.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  51 non-null     int64  
 1   arima_death_rate_mod  51 non-null     float64
 2   event_death_rate_mod  51 

# DEMOGRAPHIC VISUALIZATION

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- REQUIRES: Pop_data.csv, Marriage_data.csv ---

# Load the necessary datasets
pop_df = Pop_data
marriage_df = Marriage_data
sns.set_theme(style="whitegrid")

# --- Visualization: Population Pyramid ---
bins = [0, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84, np.inf]
labels = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+']
pop_df['age_group'] = pd.cut(pop_df['age'], bins=bins, labels=labels, right=True)
pop_pyramid = pop_df.groupby(['age_group', 'sex'], observed=True).size().unstack().fillna(0)
pop_pyramid['Male'] = -pop_pyramid.get('Male', 0)

fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(pop_pyramid.index, pop_pyramid['Male'], color='lightblue', label='Male')
ax.barh(pop_pyramid.index, pop_pyramid.get('Female', 0), color='lightpink', label='Female')
ax.set_title('Population Pyramid at Final Year', fontsize=18, weight='bold')
ax.set_xlabel('Number of People', fontsize=14)
ax.set_ylabel('Age Group', fontsize=14)
ax.legend()
ticks = ax.get_xticks()
ax.set_xticklabels([abs(int(tick)) for tick in ticks])
plt.tight_layout()
plt.savefig('demography_population_pyramid.png')
print("Generated demography_population_pyramid.png")
plt.close()

# --- Visualization: Marriages Per Year ---
marriages_by_year = marriage_df['year_of_marriage'].value_counts().sort_index()
plt.figure(figsize=(12, 7))
marriages_by_year.plot(kind='line', marker='o', linestyle='-', color='purple')
plt.title('Number of New Marriages Per Year', fontsize=18, weight='bold')
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Marriages', fontsize=14)
plt.ylim(bottom=0)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.savefig('demography_marriages_per_year.png')
print("Generated demography_marriages_per_year.png")
plt.close()

Generated demography_population_pyramid.png
Generated demography_marriages_per_year.png


In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- REQUIRES: Score_data.csv ---
score_df = Score_data
sns.set_theme(style="whitegrid")

fig, ax1 = plt.subplots(figsize=(12, 7))

color = 'tab:purple'
ax1.set_xlabel('Year', fontsize=14)
ax1.set_ylabel('Economic Index', fontsize=14, color=color)
ax1.plot(score_df['year'], score_df['economic_index'], color=color, linewidth=2.5, label='Economic Index')
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_ylim(0, 100)

ax2 = ax1.twinx()
color = 'tab:orange'
ax2.set_ylabel('Transport Index', fontsize=14, color=color)
ax2.plot(score_df['year'], score_df['transport_index'], color=color, linestyle='--', linewidth=2.5, label='Transport Index')
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0, 100)

plt.title('Calculated Score Indices Over Time', fontsize=18, weight='bold')
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
fig.tight_layout()
plt.savefig('model_score_indices.png')
print("Generated model_score_indices.png")
plt.close()

Generated model_score_indices.png


In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- REQUIRES: Rate_data.csv ---
rate_df = Rate_data
sns.set_theme(style="whitegrid")

plt.figure(figsize=(12, 7))
plt.plot(rate_df['year'], rate_df['arima_death_rate_mod'], label='ARIMA Death Rate Modifier', linestyle='--')

# You can add other modifiers as well, for example:
# plt.plot(rate_df['year'], rate_df['arima_birth_rate_mod'], label='ARIMA Birth Rate Modifier', linestyle=':')

plt.axhline(1.0, color='grey', linestyle='-', linewidth=2, label='Baseline (1.0)')
plt.title('ARIMA Modifiers Over Time', fontsize=18, weight='bold')
plt.xlabel('Year', fontsize=14)
plt.ylabel('Modifier Value', fontsize=14)
plt.legend()
plt.tight_layout()
plt.savefig('model_arima_modifiers.png')
print("Generated model_arima_modifiers.png")
plt.close()

Generated model_arima_modifiers.png


In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- REQUIRES: Trip_data.csv ---
trip_df = Trip_data
sns.set_theme(style="whitegrid")

# Pivot the data to get years as rows and purposes as columns
trip_pivot = trip_df.pivot(index='year', columns='purpose', values='count').fillna(0)

# Create the stacked area chart
plt.figure(figsize=(14, 8))
trip_pivot.plot(kind='area', stacked=True, figsize=(14, 8), colormap='viridis')

plt.title('Distribution of Commuting Purpose Per Year', fontsize=18, weight='bold')
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Trips', fontsize=14)
plt.legend(title='Purpose', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.ylim(bottom=0)
plt.tight_layout()
plt.savefig('transport_commuting_purpose.png')
print("Generated transport_commuting_purpose.png")
plt.close()

Generated transport_commuting_purpose.png


<Figure size 1400x800 with 0 Axes>

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- REQUIRES: Resource_data.csv ---
resource_df = Resource_data
sns.set_theme(style="whitegrid")

# Filter for a specific resource, for example, High School
hs_data = resource_df[resource_df['name'] == 'High School (9-12)']

plt.figure(figsize=(12, 7))
plt.plot(hs_data['year'], hs_data['capacity'], label='Capacity', color='grey', linestyle='--', linewidth=2.5)
plt.plot(hs_data['year'], hs_data['in_use'], label='In Use', color='blue', linewidth=2)
plt.plot(hs_data['year'], hs_data['waiting_or_refused'], label='Refused / Overflow', color='red', linestyle=':', linewidth=2)

plt.title('Resource Utilization: High School', fontsize=18, weight='bold')
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Students', fontsize=14)
plt.legend()
plt.ylim(bottom=0)
plt.tight_layout()
plt.savefig('resource_utilization_highschool.png')
print("Generated resource_utilization_highschool.png")
plt.close()

Generated resource_utilization_highschool.png


# COMBINED VISUALS

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fpdf import FPDF
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

# --- FILENAMES ---
# Define the exact filenames for your simulation output
POP_DATA_FILE = "mastercode02_output_poli3_20251007_2026_population_datasheet.csv"
HH_DATA_FILE = "mastercode02_output_poli3_20251007_2026_household_datasheet.csv"
ANNUAL_DATA_FILE = "mastercode02_output_poli3_20251007_2026_annual_summary.csv"
MARRIAGE_DATA_FILE = "mastercode02_output_poli3_20251007_2026_summary_marriages.csv"
RATE_DATA_FILE = "mastercode02_output_poli3_20251007_2026_summary_rates.csv"
SCORE_DATA_FILE = "mastercode02_output_poli3_20251007_2026_scores_summary.csv"
RESOURCE_DATA_FILE = "mastercode02_output_poli3_20251007_2026_summary_resources.csv"
TRIP_DATA_FILE = "mastercode02_output_poli3_20251007_2026_summary_trips.csv"
VEHICLE_DATA_FILE = "mastercode02_output_poli3_20251007_2026_summary_vehicle_events.csv"

# List to keep track of generated image files for the PDF report
generated_files = []

def create_demography_visuals(pop_data):
    """Generates all demographic visualizations."""
    print("Generating Demography visuals...")
    latest_year = pop_data['year'].max()
    df_latest = pop_data[pop_data['year'] == latest_year]

    # 1. Age Pyramid
    bins = [0, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84, np.inf]
    labels = ['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85+']
    df_latest['age_group'] = pd.cut(df_latest['age'], bins=bins, labels=labels, right=False)
    pyramid_data = df_latest.groupby(['age_group', 'sex'], observed=False).size().unstack(fill_value=0)
    pyramid_data['Male'] = -pyramid_data['Male']
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(pyramid_data.index, pyramid_data['Male'], label='Male')
    ax.barh(pyramid_data.index, pyramid_data['Female'], label='Female')
    ax.set_title(f'Age Pyramid (Year {latest_year})')
    ax.set_xlabel('Population Count')
    ax.set_ylabel('Age Group')
    ax.legend()
    ax.set_xticklabels([abs(int(x)) for x in ax.get_xticks()])
    plt.tight_layout()
    filename = 'demography_age_pyramid.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()

    # 2. Education Attainment
    edu_counts = df_latest['education_level'].value_counts()
    plt.figure(figsize=(10, 6))
    edu_counts.plot(kind='bar')
    plt.title(f'Education Attainment (Year {latest_year})')
    plt.ylabel('Population Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    filename = 'demography_education.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()

    # 3. Marital Status
    marital_counts = df_latest['marital_status'].value_counts()
    plt.figure(figsize=(8, 8))
    plt.pie(marital_counts, labels=marital_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title(f'Marital Status Composition (Year {latest_year})')
    plt.ylabel('')
    plt.tight_layout()
    filename = 'demography_marital_status.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    
    # 4. Accident Involvement by Age Group
    pop_data['age_group'] = pd.cut(pop_data['age'], bins=bins, labels=labels, right=False)
    accident_data = pop_data[pop_data['Accident Involvement'] == True]
    accident_counts = accident_data['age_group'].value_counts().sort_index()
    plt.figure(figsize=(12, 6))
    accident_counts.plot(kind='bar')
    plt.title('Accident Involvement by Age Group (All Years)')
    plt.xlabel('Age Group')
    plt.ylabel('Number of Accidents')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    filename = 'demography_accident_by_age.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    print("...done.")

def create_actuarial_visuals(annual_data, rate_data):
    """Generates all actuarial visualizations."""
    print("Generating Actuarial visuals...")
    
    # 1. Birth vs. Death Rates
    plt.figure(figsize=(10, 6))
    plt.plot(annual_data['Year'], annual_data['Birth rate'], label='Birth Rate')
    plt.plot(annual_data['Year'], annual_data['Death rate'], label='Death Rate')
    plt.title('Birth vs. Death Rates Over Time')
    plt.xlabel('Year'); plt.ylabel('Rate (per capita)'); plt.legend()
    plt.tight_layout()
    filename = 'actuarial_birth_vs_death_rates.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()

    # 2. Birth vs. Death Counts
    plt.figure(figsize=(10, 6))
    plt.plot(annual_data['Year'], annual_data['No of births'], label='Births')
    plt.plot(annual_data['Year'], annual_data['No. of Deaths'], label='Deaths')
    plt.title('Birth and Death Counts Over Time')
    plt.xlabel('Year'); plt.ylabel('Count'); plt.legend()
    plt.tight_layout()
    filename = 'actuarial_birth_vs_death_counts.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()

    # 3. ARIMA vs. Event-Driven Death Rate
    plt.figure(figsize=(12, 6))
    plt.plot(rate_data['year'], rate_data['arima_death_rate_mod'], label='ARIMA Modifier')
    plt.plot(rate_data['year'], rate_data['event_death_rate_mod'], label='Event Modifier', linestyle='--')
    plt.axhline(1.0, color='gray', linestyle=':', label='Baseline (1.0)')
    plt.title('ARIMA vs. Event-Driven Death Rate Modifiers')
    plt.xlabel('Year'); plt.ylabel('Modifier'); plt.legend()
    plt.tight_layout()
    filename = 'actuarial_arima_vs_event.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    print("...done.")

def create_household_econ_visuals(hh_data):
    """Generates all household economics visualizations."""
    print("Generating Household Economics visuals...")
    
    # 1. Household Income Distribution
    plt.figure(figsize=(10, 6))
    plt.hist(hh_data['Household Annual Income'], bins=50, edgecolor='black')
    plt.title('Household Income Distribution')
    plt.xlabel('Household Annual Income'); plt.ylabel('Frequency')
    plt.tight_layout()
    filename = 'econ_household_income_dist.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()

    # 2. Lorenz Curve & Gini Coefficient
    income = np.sort(hh_data['Household Annual Income'].values)
    cum_income = np.cumsum(income)
    lorenz_curve = cum_income / cum_income[-1]
    gini = 1 - 2 * np.trapz(lorenz_curve, dx=1/len(lorenz_curve))
    plt.figure(figsize=(8, 8))
    plt.plot([0, 1], [0, 1], 'k:', label='Line of Perfect Equality')
    plt.plot(np.linspace(0, 1, len(lorenz_curve)), lorenz_curve, label='Lorenz Curve')
    plt.title(f'Lorenz Curve of Income Inequality (Gini: {gini:.3f})')
    plt.xlabel('Cumulative Share of Households'); plt.ylabel('Cumulative Share of Income')
    plt.legend(); plt.tight_layout()
    filename = 'econ_lorenz_curve.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    
    # 3. Net Income Minus Cost
    plt.figure(figsize=(10, 6))
    plt.hist(hh_data['Net_income_minus_cost'], bins=50, edgecolor='black')
    plt.axvline(0, color='red', linestyle='--', label='Breakeven Point')
    plt.title('Household Financial Surplus/Deficit')
    plt.xlabel('Net Income - Required Cost'); plt.ylabel('Frequency'); plt.legend()
    plt.tight_layout()
    filename = 'econ_net_income_minus_cost.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()

    # 4. Savings vs. Loan Balance
    plt.figure(figsize=(10, 10))
    plt.scatter(hh_data['Loan Balance'], hh_data['Savings Balance'], alpha=0.5)
    plt.plot([0, hh_data['Loan Balance'].max()], [0, hh_data['Loan Balance'].max()], 'r--', label='Savings = Loans')
    plt.title('Household Savings vs. Loan Balance')
    plt.xlabel('Loan Balance'); plt.ylabel('Savings Balance'); plt.legend()
    plt.xscale('log'); plt.yscale('log')
    plt.tight_layout()
    filename = 'econ_savings_vs_loan.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    print("...done.")

def create_macro_econ_visuals(annual_data, rate_data, score_data):
    """Generates all macro-economic visualizations."""
    print("Generating Macro-Economic visuals...")
    
    # 1. Employment Rate
    plt.figure(figsize=(10, 6))
    plt.plot(annual_data['Year'], annual_data['Employment Rate'])
    plt.title('Employment Rate Over Time'); plt.xlabel('Year'); plt.ylabel('Rate')
    plt.ylim(0, 1); plt.tight_layout()
    filename = 'macro_employment_rate.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    
    # 2. Income vs. Cost Trends
    plt.figure(figsize=(10, 6))
    plt.plot(annual_data['Year'], annual_data['Avg Annual Income'], label='Avg Annual Income')
    plt.plot(annual_data['Year'], annual_data['Avg Annual Req Cost'], label='Avg Required Cost')
    plt.title('Average Income vs. Required Cost'); plt.xlabel('Year'); plt.ylabel('USD ($)'); plt.legend()
    plt.tight_layout()
    filename = 'macro_income_vs_cost.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()

    # 3. Inflation Tracks
    plt.figure(figsize=(12, 6))
    plt.plot(rate_data['year'], rate_data['salary_inflation'], label='Salary Inflation')
    plt.plot(rate_data['year'], rate_data['cpi_inflation'], label='CPI Inflation (from ARIMA)')
    plt.title('Inflation Tracks'); plt.xlabel('Year'); plt.ylabel('Rate'); plt.legend()
    plt.tight_layout()
    filename = 'macro_inflation.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    
    # 4. Scenario Scores
    plt.figure(figsize=(12, 6))
    plt.plot(score_data['year'], score_data['economic_index'], label='Economic Index')
    plt.plot(score_data['year'], score_data['transport_index'], label='Transport Index')
    plt.title('Scenario Score Indices Over Time'); plt.xlabel('Year'); plt.ylabel('Index Score (0-100)'); plt.legend()
    plt.ylim(0, 100); plt.tight_layout()
    filename = 'macro_scenario_scores.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    print("...done.")
    
def create_mobility_visuals(annual_data, trip_data, vehicle_data, resource_data):
    """Generates all mobility and traffic visualizations."""
    print("Generating Mobility & Traffic visuals...")

    # 1. Cars per 1000 People
    annual_data['cars_per_1000'] = (annual_data['Total Number of Cars'] / annual_data['Population']) * 1000
    plt.figure(figsize=(10, 6))
    plt.plot(annual_data['Year'], annual_data['cars_per_1000'])
    plt.title('Cars per 1,000 People'); plt.xlabel('Year'); plt.ylabel('Cars per 1,000 People')
    plt.ylim(bottom=0); plt.tight_layout()
    filename = 'mobility_cars_per_1000.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()

    # 2. Commuting Purpose Mix
    trip_pivot = trip_data.pivot(index='year', columns='purpose', values='count').fillna(0)
    trip_pivot.plot(kind='area', stacked=True, figsize=(12, 7))
    plt.title('Purpose of Commutation Mix Over Time'); plt.xlabel('Year'); plt.ylabel('Number of Trips')
    plt.legend(title='Purpose', bbox_to_anchor=(1.05, 1), loc='upper left'); plt.tight_layout()
    filename = 'mobility_trip_purpose_mix.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    
    # 3. Top Vehicle Events
    top_events = vehicle_data['event'].value_counts().nlargest(5).index
    event_pivot = vehicle_data.pivot_table(index='year', columns='event', aggfunc='size', fill_value=0)
    plt.figure(figsize=(12, 7))
    event_pivot[top_events].plot(figsize=(12, 7))
    plt.title('Top 5 Vehicle Events Over Time'); plt.xlabel('Year'); plt.ylabel('Count')
    plt.legend(title='Event Type'); plt.tight_layout()
    filename = 'mobility_vehicle_events.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()

    # 4. Resource Utilization (Roads)
    road_data = resource_data[resource_data['name'].str.contains('Road:')]
    road_pivot = road_data.pivot(index='year', columns='name', values='utilization')
    road_pivot.plot(figsize=(12, 7))
    plt.title('Road Network Utilization Over Time'); plt.xlabel('Year'); plt.ylabel('Utilization (Load Factor)')
    plt.axhline(1.0, color='red', linestyle='--', label='Gridlock Threshold')
    plt.axhline(0.7, color='orange', linestyle=':', label='Congestion Threshold')
    plt.legend(); plt.tight_layout()
    filename = 'mobility_road_utilization.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    print("...done.")

def create_bonus_visuals(marriage_data):
    """Generates bonus demographic flow visualizations."""
    print("Generating Bonus visuals...")
    
    # 1. Marriage Ages
    plt.figure(figsize=(10, 8))
    plt.hist2d(marriage_data['man_age'], marriage_data['woman_age'], bins=(20, 20), cmap='magma')
    plt.colorbar(label='Number of Marriages')
    plt.title('Joint Distribution of Marriage Ages'); plt.xlabel("Man's Age at Marriage"); plt.ylabel("Woman's Age at Marriage")
    plt.tight_layout()
    filename = 'bonus_marriage_ages.png'
    plt.savefig(filename); generated_files.append(filename); plt.close()
    print("...done.")

def generate_pdf_report(image_files):
    """Bundles all generated PNGs into a single PDF report."""
    print(f"\nGenerating PDF report from {len(image_files)} images...")
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    for image in image_files:
        pdf.add_page()
        pdf.set_font('Arial', 'B', 16)
        # Clean up filename for title
        title = image.replace('.png', '').replace('_', ' ').title()
        pdf.cell(0, 10, title, 0, 1, 'C')
        # A4 page width is 210mm. Margins are 10mm each side. Image width = 190.
        pdf.image(image, x=10, y=30, w=190)
    
    pdf_filename = 'Simulation_Visualization_Report.pdf'
    pdf.output(pdf_filename, 'F')
    print(f"...SUCCESS! Report saved as {pdf_filename}")

def main():
    """Main function to load data and generate all visualizations."""
    try:
        # Load all dataframes
        pop_data = pd.read_csv(POP_DATA_FILE)
        hh_data = pd.read_csv(HH_DATA_FILE)
        annual_data = pd.read_csv(ANNUAL_DATA_FILE)
        marriage_data = pd.read_csv(MARRIAGE_DATA_FILE)
        rate_data = pd.read_csv(RATE_DATA_FILE)
        score_data = pd.read_csv(SCORE_DATA_FILE)
        resource_data = pd.read_csv(RESOURCE_DATA_FILE)
        trip_data = pd.read_csv(TRIP_DATA_FILE)
        vehicle_data = pd.read_csv(VEHICLE_DATA_FILE)
        print("--- All data files loaded successfully. ---")
    except FileNotFoundError as e:
        print(f"ERROR: Could not find a required data file: {e.filename}")
        print("Please ensure all CSV files are in the same directory as this script.")
        return

    # Generate visuals by theme
    create_demography_visuals(pop_data)
    create_actuarial_visuals(annual_data, rate_data)
    create_household_econ_visuals(hh_data)
    create_macro_econ_visuals(annual_data, rate_data, score_data)
    create_mobility_visuals(annual_data, trip_data, vehicle_data, resource_data)
    create_bonus_visuals(marriage_data)
    
    # Generate the final PDF report
    generate_pdf_report(generated_files)


if __name__ == '__main__':
    main()


--- All data files loaded successfully. ---
Generating Demography visuals...
...done.
Generating Actuarial visuals...
...done.
Generating Household Economics visuals...
...done.
Generating Macro-Economic visuals...
...done.
Generating Mobility & Traffic visuals...
...done.
Generating Bonus visuals...
...done.

Generating PDF report from 20 images...
...SUCCESS! Report saved as Simulation_Visualization_Report.pdf


<Figure size 1200x700 with 0 Axes>

In [46]:
pip install FPDF

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for FPDF: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40713 sha256=390b60178f8e2bece61982904be6b1d0166e799f386b928b9a2b98ae8b2acdfb
  Stored in directory: c:\users\lord of lords\appdata\local\pip\cache\wheels\6e\62\11\dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built FPDF
Note: you may need to restart the kernel to use updated packages.


# RESOURCES VISUALIZATION

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

# --- FILENAME ---
RESOURCE_DATA_FILE = "mastercode02_output_poli3_20251007_2026_summary_resources.csv"

def plot_education_resources(df):
    """Visualizes capacity vs. usage for key educational institutions."""
    print("Generating Education resource visuals...")
    edu_df = df[df['name'].isin(['High School (9-12)', 'University', 'Community College'])]
    
    fig, axes = plt.subplots(3, 1, figsize=(12, 18), sharex=True)
    fig.suptitle('Education Resource Utilization Over Time', fontsize=20, weight='bold')

    # Define a color palette
    colors = {'High School (9-12)': 'blue', 'University': 'green', 'Community College': 'purple'}

    for i, name in enumerate(colors.keys()):
        ax = axes[i]
        subset = edu_df[edu_df['name'] == name]
        ax.plot(subset['year'], subset['capacity'], label='Capacity', color='black', linestyle='--', linewidth=2)
        ax.plot(subset['year'], subset['in_use'], label='In Use (Enrollment)', color=colors[name], linewidth=2.5)
        ax.fill_between(subset['year'], subset['in_use'], color=colors[name], alpha=0.2)
        
        ax.set_title(name, fontsize=14)
        ax.set_ylabel('Number of Students')
        ax.legend()
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax.set_ylim(bottom=0)

    axes[-1].set_xlabel('Year')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig('resources_education.png')
    plt.close()
    print("...done.")

def plot_employer_resources(df):
    """Visualizes capacity vs. usage for major employers."""
    print("Generating Employer resource visuals...")
    # Select top 4 employers by capacity for clarity
    employer_df = df[df['name'].isin(['Healthcare System', 'Higher Education', 'City & County Gov.', 'Retail & Hospitality'])]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10), sharex=True, sharey=True)
    fig.suptitle('Major Employer Job Capacity vs. Filled Positions', fontsize=20, weight='bold')
    axes = axes.flatten()

    colors = {'Healthcare System': 'red', 'Higher Education': 'orange', 'City & County Gov.': 'brown', 'Retail & Hospitality': 'teal'}

    for i, name in enumerate(colors.keys()):
        ax = axes[i]
        subset = employer_df[employer_df['name'] == name]
        ax.plot(subset['year'], subset['capacity'], label='Capacity (Jobs)', color='black', linestyle='--', linewidth=2)
        ax.plot(subset['year'], subset['in_use'], label='In Use (Employees)', color=colors[name], linewidth=2.5)
        ax.fill_between(subset['year'], subset['in_use'], color=colors[name], alpha=0.2)
        
        ax.set_title(name, fontsize=14)
        if i % 2 == 0:
            ax.set_ylabel('Number of Positions')
        if i >= 2:
            ax.set_xlabel('Year')
        ax.legend()
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax.set_ylim(bottom=0)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig('resources_employers.png')
    plt.close()
    print("...done.")

def plot_financial_resources(df):
    """Visualizes the utilization percentage of city-wide financial caps."""
    print("Generating Financial resource visuals...")
    fin_df = df[df['name'].isin(['Bank (Savings)', 'Bank (Loan)', 'Government Support'])]
    
    # Pivot data to get years as index and resource names as columns
    fin_pivot = fin_df.pivot(index='year', columns='name', values='utilization') * 100
    
    fin_pivot.plot(kind='bar', stacked=True, figsize=(16, 8), colormap='viridis', width=0.8)
    
    plt.title('Annual Utilization of Financial Resources', fontsize=20, weight='bold')
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Utilization of Annual Capacity (%)', fontsize=14)
    plt.xticks(rotation=45)
    plt.legend(title='Resource Type')
    plt.grid(axis='y', linestyle='--', linewidth=0.5)
    
    # Set x-axis ticks to be less crowded
    ax = plt.gca()
    ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True, nbins=20))
    
    plt.tight_layout()
    plt.savefig('resources_financial.png')
    plt.close()
    print("...done.")

def plot_transport_resources(df):
    """Visualizes the utilization of roads and public transit."""
    print("Generating Transportation resource visuals...")
    transport_df = df[df['name'].str.contains('Road:|Bus Service')]
    
    # Pivot data for plotting
    transport_pivot = transport_df.pivot(index='year', columns='name', values='utilization')

    plt.figure(figsize=(14, 8))
    for column in transport_pivot.columns:
        plt.plot(transport_pivot.index, transport_pivot[column], label=column, linewidth=2.5)

    # Add reference lines for traffic congestion and gridlock
    plt.axhline(1.0, color='red', linestyle='--', linewidth=1.5, label='Gridlock Threshold (100%)')
    plt.axhline(0.7, color='orange', linestyle=':', linewidth=1.5, label='Congestion Threshold (70%)')

    plt.title('Transportation Network Utilization Over Time', fontsize=20, weight='bold')
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Utilization (%)', fontsize=14)
    plt.legend(title='Resource')
    plt.ylim(0, max(1.1, transport_pivot.max().max() * 1.05)) # Ensure y-axis covers at least 110%
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    
    plt.tight_layout()
    plt.savefig('resources_transportation.png')
    plt.close()
    print("...done.")


def main():
    """Main function to load data and generate all resource visualizations."""
    try:
        resource_data = pd.read_csv(RESOURCE_DATA_FILE)
        print("--- Resource data file loaded successfully. ---")
    except FileNotFoundError:
        print(f"ERROR: Could not find the required data file: {RESOURCE_DATA_FILE}")
        print("Please ensure the CSV file is in the same directory as this script.")
        return

    # Generate visuals by resource category
    plot_education_resources(resource_data)
    plot_employer_resources(resource_data)
    plot_financial_resources(resource_data)
    plot_transport_resources(resource_data)
    
    print("\n--- All resource visualizations have been generated. ---")


if __name__ == '__main__':
    main()


--- Resource data file loaded successfully. ---
Generating Education resource visuals...
...done.
Generating Employer resource visuals...
...done.
Generating Financial resource visuals...
...done.
Generating Transportation resource visuals...
...done.

--- All resource visualizations have been generated. ---


# POPULATION BY NUMBER OF CARS OVER TIME

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

# --- FILENAME ---
ANNUAL_DATA_FILE = "mastercode02_output_poli3_20251007_2026_annual_summary.csv"

def create_population_vs_cars_chart(df):
    """
    Generates a time trend graph comparing the total population to the total number of cars,
    using a dual-axis plot for clarity.
    """
    print("Generating population vs. number of cars chart...")

    # 1. Set up the figure and primary axis
    fig, ax1 = plt.subplots(figsize=(14, 8))

    # 2. Plot Population on the primary (left) y-axis
    color1 = 'tab:blue'
    ax1.set_xlabel('Year', fontsize=14)
    ax1.set_ylabel('Total Population', fontsize=14, color=color1)
    ax1.plot(df['Year'], df['Population'], color=color1, linewidth=3, label='Population')
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

    # 3. Create a secondary (right) y-axis that shares the same x-axis
    ax2 = ax1.twinx()

    # 4. Plot Total Number of Cars on the secondary y-axis
    color2 = 'tab:red'
    ax2.set_ylabel('Total Number of Cars', fontsize=14, color=color2)
    ax2.plot(df['Year'], df['Total Number of Cars'], color=color2, linewidth=3, linestyle='--', label='Number of Cars')
    ax2.tick_params(axis='y', labelcolor=color2)

    # 5. Formatting and labels
    plt.title('Population Growth vs. Car Ownership Over Time', fontsize=20, weight='bold')
    
    # Add a unified legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=12)

    plt.tight_layout()
    plt.savefig('population_vs_cars_trend.png')
    plt.close()

    print("...done. Chart saved as population_vs_cars_trend.png")

def main():
    """Main function to load data and generate the visualization."""
    try:
        annual_data = pd.read_csv(ANNUAL_DATA_FILE)
        print("--- Annual summary data file loaded successfully. ---")
    except FileNotFoundError:
        print(f"ERROR: Could not find the required data file: {ANNUAL_DATA_FILE}")
        print("Please ensure the CSV file is in the same directory as this script.")
        return

    create_population_vs_cars_chart(annual_data)

if __name__ == '__main__':
    main()


--- Annual summary data file loaded successfully. ---
Generating population vs. number of cars chart...
...done. Chart saved as population_vs_cars_trend.png
